# AutoGrad: 파이토치에서 신경망모형(딥러닝 모형)의 파라미터(가중치)를 Back Propagation(역전파 알고리즘) 방법으로 갱신하는 방식 
# 하이퍼 파라미터: Batch Size(한 번의 학습에 이용되는 데이터의 개수), Input Size(입력층의 크기), Hidden Size(은닉층의 크기), Output Size(출력층의 크기) 

In [18]:
import torch

if torch.cuda.is_available():
    print("1")
    DEVICE = torch.device('cuda')
else :
    print("2")
    DEVICE = torch.device('cpu')

1


In [19]:
BATCH_SIZE = 64
INPUT_SIZE = 1000
HIDDEN_SIZE = 100
OUTPUT_SIZE = 10
print(BATCH_SIZE)
print(INPUT_SIZE)
print(HIDDEN_SIZE)
print(OUTPUT_SIZE)

64
1000
100
10


In [20]:
x=torch.randn(BATCH_SIZE,INPUT_SIZE,device=DEVICE,dtype=torch.float,requires_grad=False)
print(x)

tensor([[-0.9475, -1.2196,  0.5588,  ...,  0.0072, -0.4705,  0.4033],
        [ 0.1483,  0.3810,  1.7937,  ...,  0.6912, -0.7019, -0.6337],
        [-0.2833, -0.9163,  1.8123,  ..., -0.3540, -0.1596,  0.0713],
        ...,
        [ 0.5813, -0.9443,  0.0976,  ..., -1.2587, -0.3046,  0.3054],
        [ 0.9692,  0.3718, -0.8067,  ..., -1.2300,  0.4644, -1.2584],
        [ 0.3934, -1.3875,  1.2164,  ...,  0.8546, -0.1714,  1.0588]],
       device='cuda:0')


In [21]:
y=torch.randn(BATCH_SIZE,OUTPUT_SIZE,device=DEVICE,dtype=torch.float,requires_grad=False)
print(y)

tensor([[-0.8540,  0.9143, -2.1336, -1.3255, -0.2405,  1.1147,  0.9978, -0.6182,
          0.4226,  0.4867],
        [-0.8082,  0.3108,  2.5887, -0.8549,  0.3806, -0.4469,  1.1943, -0.4710,
          0.8002, -0.7782],
        [ 0.0198, -0.7748, -0.0810,  1.7145, -1.0899, -1.8613,  0.3350,  0.0717,
         -2.1250, -1.5121],
        [-0.3613,  0.7236,  1.1295,  0.2271,  1.9148,  0.2549,  0.1985,  1.4636,
         -0.5668,  0.3253],
        [ 0.2811, -0.3515,  0.3438, -0.5770, -0.9852, -0.9644,  0.4768, -1.4826,
         -1.1145,  1.6074],
        [ 0.3234,  1.1868,  0.3058, -1.0391,  0.3467, -0.3378, -2.1136,  0.4759,
          0.4172,  0.8899],
        [ 3.0393, -0.1564, -1.4453, -0.4229,  0.2675,  0.2331,  1.1247, -0.4028,
         -2.6947, -1.3814],
        [-0.2087,  0.1656, -0.1115, -0.3099, -0.0302, -0.9190,  1.2867,  0.0314,
         -0.6601,  0.5374],
        [ 0.0381, -1.0460,  0.2529,  0.1318,  0.8136,  0.0811, -0.0423,  0.0612,
          0.3324, -0.6481],
        [-1.6194,  

In [22]:
w1 = torch.randn(INPUT_SIZE,HIDDEN_SIZE,device=DEVICE,dtype=torch.float,requires_grad=True)
print(w1)

tensor([[ 0.0820, -1.0758,  0.3162,  ..., -0.1407,  0.0738, -0.0180],
        [ 0.2754, -0.9168,  0.2948,  ...,  0.5430, -1.3371, -0.2618],
        [ 1.1685,  0.3495, -0.0147,  ...,  0.4347,  0.0519,  0.1590],
        ...,
        [-0.4302, -0.5067, -1.4232,  ..., -0.8872, -2.6132, -2.2579],
        [ 0.9312,  0.0295,  0.1922,  ..., -0.2986,  0.1719, -1.3953],
        [-0.9810,  0.5395, -0.9410,  ...,  1.4403,  0.3743, -0.2304]],
       device='cuda:0', requires_grad=True)


In [23]:
w2 = torch.randn(HIDDEN_SIZE,OUTPUT_SIZE,device=DEVICE,dtype=torch.float,requires_grad=True)
print(w2)

tensor([[ 1.3097e+00,  1.3090e+00,  2.5490e-01,  1.6680e+00,  2.6304e-01,
          1.8100e+00, -4.1314e-01, -7.4181e-01, -3.7094e-02,  2.2484e-01],
        [-1.8602e-01,  7.3684e-02, -1.4872e+00, -8.3337e-01,  2.2989e-01,
          1.5122e+00, -1.0291e+00, -9.5394e-01, -1.3699e+00, -7.6529e-01],
        [-1.4720e+00, -1.9297e+00, -8.6646e-02, -7.3524e-01,  1.0748e+00,
         -1.1167e-01,  1.3264e-01,  1.2475e-01,  9.4693e-01, -6.8157e-01],
        [ 8.0520e-01,  5.6985e-01,  1.3907e+00, -7.3188e-02,  9.8270e-01,
          1.5217e+00,  9.7458e-01, -1.6122e+00,  1.6489e+00,  8.6236e-01],
        [-3.2491e-01,  1.4610e-01,  3.1998e-01, -3.0910e-01, -1.1445e+00,
         -8.3408e-01,  1.7140e-01,  1.0268e+00, -1.8129e+00, -7.2382e-01],
        [-1.2877e+00, -1.6147e+00, -6.5345e-01, -1.1069e+00,  8.8431e-01,
         -2.3325e-01, -9.0255e-01,  1.5560e+00, -5.5289e-01,  8.6674e-01],
        [ 2.6234e+00,  1.0290e+00, -8.1767e-01, -6.0651e-02,  1.9303e+00,
         -3.3055e-01, -1.6585e+0

In [24]:
learning_rate=1e-6
for t in range(1,501):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss =(y_pred-y).pow(2).sum()
    if t % 100 == 0:
        print(f"Iteration:{t} \t Loss: ", loss.item())
    loss.backward()
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_() 

Iteration:100 	 Loss:  548.0221557617188
Iteration:200 	 Loss:  2.5163841247558594
Iteration:300 	 Loss:  0.019926689565181732
Iteration:400 	 Loss:  0.0004400567850098014
Iteration:500 	 Loss:  6.224481330718845e-05
